In [1]:
"""
作者：librauee
微信公众号：老肥码码码
日期：2020.12.10
"""

'\n作者：librauee\n微信公众号：老肥码码码\n日期：2020.12.10\n'

In [1]:
import pandas as pd
pd.set_option('max_columns', 300)
from tqdm import tqdm
import warnings
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

warnings.filterwarnings('ignore')
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [2]:
# 目标编码
def kfold_mean(df_train, df_test, target, target_mean_list):
    folds = StratifiedKFold(n_splits=5)     #使用5折交叉验证.先获取了5折数据各自的索引

    mean_of_target = df_train[target].mean()

    for fold_, (trn_idx, val_idx) in tqdm(
            enumerate(folds.split(df_train, y=df_train['label']))):
        tr_x = df_train.iloc[trn_idx, :]
        vl_x = df_train.iloc[val_idx, :]

        for col in target_mean_list:
            df_train.loc[vl_x.index, f'{col}_target_enc'] = vl_x[col].map(
                tr_x.groupby(col)[target].mean())

    for col in target_mean_list:
        df_train[f'{col}_target_enc'].fillna(mean_of_target, inplace=True)

        df_test[f'{col}_target_enc'] = df_test[col].map(
            df_train.groupby(col)[f'{col}_target_enc'].mean())

        df_test[f'{col}_target_enc'].fillna(mean_of_target, inplace=True)
    return pd.concat([df_train, df_test], ignore_index=True)

feature_list =  ['HYZK', 'ZHIYE', 'ZHICHEN', 'ZHIWU', 'XUELI', 'DWJJLX', 'DWSSHY', 'GRZHZT'
                   ]
data = pd.concat([train, test], ignore_index=True)
data = kfold_mean(data[~data['label'].isna()], data[data['label'].isna()],
                  'label',
                  feature_list)

5it [00:00, 27.39it/s]


In [3]:
data.head(10)

,id,XINGBIE,CSNY,HYZK,ZHIYE,ZHICHEN,ZHIWU,XUELI,DWJJLX,DWSSHY,GRJCJS,GRZHZT,GRZHYE,GRZHSNJZYE,GRZHDNGJYE,GRYJCE,DWYJCE,DKFFE,DKYE,DKLL,label,HYZK_target_enc,ZHIYE_target_enc,ZHICHEN_target_enc,ZHIWU_target_enc,XUELI_target_enc,DWJJLX_target_enc,DWSSHY_target_enc,GRZHZT_target_enc
0,train_0,1,1038672000,90,90,999,0,99,150,12,1737.0,1,3223.515,801.310,837.000,312.00,312.00,175237,154112.935,2.708,0.0,0.068955,0.068843,0.068681,0.068944,0.068944,0.024447,0.061192,0.064913
1,train_1,2,504892800,90,90,999,0,99,110,0,4894.0,1,18055.195,53213.220,1065.200,795.84,795.84,300237,298252.945,2.979,0.0,0.068955,0.068843,0.068681,0.068944,0.068944,0.093762,0.016744,0.064913
2,train_2,1,736185600,90,90,999,0,99,150,9,10297.0,1,27426.600,13963.140,7230.020,1444.20,1444.20,150237,147339.130,2.708,0.0,0.068955,0.068843,0.068681,0.068944,0.068944,0.024447,0.044090,0.064913
3,train_3,1,428515200,90,90,999,0,99,150,7,10071.5,1,111871.130,99701.265,2271.295,1417.14,1417.14,350237,300653.780,2.708,0.0,0.068955,0.068843,0.068681,0.068944,0.068944,0.024447,0.012766,0.064913
4,train_4,2,544204800,90,90,999,0,99,900,14,2007.0,1,237.000,11028.875,35.780,325.50,325.50,150237,145185.010,2.708,0.0,0.068955,0.068843,0.068681,0.068944,0.068944,0.112285,0.077085,0.064913
5,train_5,1,488995200,90,90,999,0,99,150,14,1192.0,1,9648.315,7388.550,771.800,303.85,303.85,150237,149743.210,2.708,0.0,0.068955,0.068843,0.068681,0.068944,0.068944,0.024447,0.077085,0.064913
6,train_6,1,562694400,90,90,999,0,99,160,2,7297.0,1,22846.975,12179.565,5885.000,801.80,801.80,150237,146723.725,2.708,0.0,0.068955,0.068843,0.068681,0.068944,0.068944,0.020433,0.023810,0.064913
7,train_7,1,1091289600,90,90,999,0,99,150,7,1399.0,1,1808.135,6405.055,1050.400,318.34,318.34,125237,64938.955,2.708,0.0,0.068955,0.068843,0.068681,0.068944,0.068944,0.024447,0.012766,0.064913
8,train_8,2,615222000,90,90,999,0,99,150,9,6408.0,1,79304.635,52365.820,6161.160,977.52,977.52,300237,270579.955,2.979,0.0,0.068955,0.068843,0.068681,0.068944,0.068944,0.024447,0.044090,0.064913
9,train_9,2,454867200,90,90,999,0,99,150,14,3573.5,1,2009.000,61126.175,536.750,637.38,637.38,230237,142296.640,2.708,0.0,0.068955,0.068843,0.068681,0.068944,0.068944,0.024447,0.077085,0.064913


In [4]:
# 频数统计
cat_col = ['HYZK', 'ZHIYE', 'ZHICHEN', 'ZHIWU', 'XUELI', 'DWJJLX', 'DWSSHY', 'GRZHZT']
for col in cat_col:
    data[col + '_COUNT'] = data[col].map(data[col].value_counts())
    col_idx = data[col].value_counts()
    for idx in col_idx[col_idx < 10].index:
        data[col] = data[col].replace(idx, -1)  

In [5]:
# 偏离值特征
group_list = ['HYZK', 'ZHIYE', 'ZHICHEN', 'ZHIWU', 'XUELI', 'DWJJLX', 'DWSSHY', 'GRZHZT']
num_feature_list = ['GRYJCE', 'DKFFE', 'DKLL', 'DKYE', 'GRJCJS', 'GRZHSNJZYE', 'GRZHDNGJYE']                   
for group in group_list:
    for feature in num_feature_list:
        tmp = data.groupby(group)[feature].agg([sum, min, max, np.mean]).reset_index()
        tmp = pd.merge(data, tmp, on=group, how='left')
        data['{}-mean_gb_{}'.format(feature, group)] = data[feature] - tmp['mean']
        data['{}-min_gb_{}'.format(feature, group)] = data[feature] - tmp['min']
        data['{}-max_gb_{}'.format(feature, group)] = data[feature] - tmp['max']
        data['{}/sum_gb_{}'.format(feature, group)] = data[feature] / tmp['sum']  

In [6]:
data.shape

(55000, 261)

In [7]:
X_train, X_test = data[~data['label'].isna()], data[data['label'].isna()]
y = X_train['label']
drop_features = ['label', 'id', 'CSNY']

X_train = X_train.drop(drop_features, axis=1)
X_test = X_test.drop(drop_features, axis=1)

cat_col = ['HYZK', 'ZHIYE', 'ZHICHEN', 'ZHIWU', 'XUELI', 'DWJJLX', 'DWSSHY', 'GRZHZT']
X_train[cat_col] = X_train[cat_col].astype('category')
X_test[cat_col] = X_test[cat_col].astype('category')

In [8]:
# 评价指标
def tpr_weight_funtion(y_true,y_predict):

    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3

In [9]:
# 训练&预测
KF = StratifiedKFold(n_splits=5, random_state=723)
params = {
    'verbose':-1, 
    'objective':'binary',
    'metric':'auc',
    'num_iterations': 10000, 
}


oof_lgb = np.zeros(len(X_train))
predictions_lgb = np.zeros((len(X_test)))

for fold_, (trn_idx, val_idx) in enumerate(KF.split(X_train.values, y.values)):
    print("fold n°{}".format(fold_))
    print('trn_idx:',trn_idx)
    print('val_idx:',val_idx)
    trn_data = lgb.Dataset(X_train.iloc[trn_idx],label=y.iloc[trn_idx])    
    val_data = lgb.Dataset(X_train.iloc[val_idx],label=y.iloc[val_idx])
    num_round = 10000
    clf = lgb.train(
        params,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=500,
                    early_stopping_rounds=200,  
        categorical_feature=cat_col
    )
        
    oof_lgb[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=clf.best_iteration)
    predictions_lgb[:] += clf.predict(X_test, num_iteration=clf.best_iteration) 
print("AUC score: {}".format(roc_auc_score(y, oof_lgb)))
print("TPR weight: {}".format(tpr_weight_funtion(y, oof_lgb)))

ValueError: Setting a random_state has no effect since shuffle is False. You should leave random_state to its default (None), or set shuffle=True.

In [11]:
submit = pd.read_csv('./data/submit.csv')
submit['label'] = predictions_lgb / 5
submit.to_csv('submit_baseline.csv', index=False)